In [1]:
from src.dataloaders.sources.synth_labor import SyntheticLaborSource

In [2]:
## How to create a SOURCE!
r = SyntheticLaborSource().prepare()

In [1]:
from src.dataloaders.population.users import UserPopulation

In [2]:
users = UserPopulation()

In [3]:
users.data_split()

DataSplit(train=array([ 60,  22,  57,  19,  34,  43,  51,  28,  93,  72,   3,  26,  25,
        53,  82, 100,  52,  62,   5,  98,  95,  38,  29,  81,   8,  58,
        80,  27,  77,  79,  39,   1,  87,  47,  11,  30,  71,  46,  18,
        40,  92,  41,  90,   6,  69,   2,  96,  85,  64,   4,  84,  94,
        21,  88,  44,  74,  33,  76,  24,  97,  70,  16,  49,  56,  61,
        10,  63,  32,  73,  17]), val=array([45, 83, 68, 35, 31, 59, 75, 89, 99, 55, 50,  7, 91, 12, 86]), test=array([42, 23, 20, 36, 65, 48, 13, 54, 15, 14, 67, 37, 66, 78,  9]))